In [1]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
pd.set_option('display.max_columns', None)

In [2]:
new_shorelines = pd.concat(gpd.read_file(f) for f in glob("nelson*.geojson")).reset_index(drop=True)
new_shorelines

,geometry
0,"LINESTRING (173.01824 -41.01944, 173.01826 -41..."
1,"LINESTRING (173.47974 -41.15699, 173.4774 -41...."
2,"LINESTRING (173.02198 -41.03365, 173.02163 -41..."
3,"LINESTRING (173.0198 -41.02696, 173.01915 -41...."
4,"LINESTRING (172.76125 -40.7872, 172.75709 -40...."
5,"LINESTRING (173.08833 -41.23502, 173.08758 -41..."
6,"LINESTRING (172.87564 -40.83355, 172.87389 -40..."
7,"LINESTRING (173.06855 -41.1907, 173.06795 -41...."
8,"LINESTRING (173.06438 -41.18284, 173.06311 -41..."
9,"LINESTRING (172.69671 -40.73009, 172.69534 -40..."


In [3]:
m = new_shorelines.explore(tiles="ESRI.WorldImagery")
gpd.GeoSeries(new_shorelines.geometry.apply(lambda line: Point(line.coords[0])), crs=new_shorelines.crs).explore(m=m, color="red", name="transect start")
m

In [4]:
shorelines = gpd.read_file("shorelines.geojson")
transects = gpd.read_file("transects_extended.geojson")
poly = gpd.read_file("polygons.geojson")

In [10]:
nz_shorelines = shorelines[shorelines.id.str.startswith("nzd")]
latest_siteid = nz_shorelines.id.max()
latest_siteid_int = int(latest_siteid[4:])
new_siteids = new_shorelines.index.to_series().apply(lambda i: f"nzd0{latest_siteid_int + i + 1}")
print(f"Latest siteid is {latest_siteid}, so the new sites will be {new_siteids}")
new_shorelines["id"] = new_siteids
new_shorelines

Latest siteid is nzd0564, so the new sites will be 0     nzd0565
1     nzd0566
2     nzd0567
3     nzd0568
4     nzd0569
5     nzd0570
6     nzd0571
7     nzd0572
8     nzd0573
9     nzd0574
10    nzd0575
11    nzd0576
12    nzd0577
13    nzd0578
14    nzd0579
dtype: object


,geometry,id
0,"LINESTRING (173.01824 -41.01944, 173.01826 -41...",nzd0565
1,"LINESTRING (173.47974 -41.15699, 173.4774 -41....",nzd0566
2,"LINESTRING (173.02198 -41.03365, 173.02163 -41...",nzd0567
3,"LINESTRING (173.0198 -41.02696, 173.01915 -41....",nzd0568
4,"LINESTRING (172.76125 -40.7872, 172.75709 -40....",nzd0569
5,"LINESTRING (173.08833 -41.23502, 173.08758 -41...",nzd0570
6,"LINESTRING (172.87564 -40.83355, 172.87389 -40...",nzd0571
7,"LINESTRING (173.06855 -41.1907, 173.06795 -41....",nzd0572
8,"LINESTRING (173.06438 -41.18284, 173.06311 -41...",nzd0573
9,"LINESTRING (172.69671 -40.73009, 172.69534 -40...",nzd0574


In [12]:
pd.concat((shorelines, new_shorelines)).to_file("shorelines.geojson", driver="GeoJSON")

In [13]:
new_polys = gpd.GeoDataFrame({"id": new_siteids}, geometry=new_shorelines.to_crs(2193).buffer(100).minimum_rotated_rectangle().to_crs(4326), crs=4326)
new_polys.explore()

In [14]:
pd.concat((poly, new_polys)).to_file("polygons.geojson", driver="GeoJSON")

In [15]:
nz_transects = transects[transects.id.str.startswith("nzd")].to_crs(2193)
nz_transects

,id,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
95050,nzd0001-0000,nzd0001,359.037136,3197.737936,1.000000,0.050,0.0397,0.0679,-0.048007,194.0,152.0,0.000243,19.220344,557.970123,23.621391,324.525695,None,"LINESTRING (1596662.375 6190263.089, 1596646.6..."
95051,nzd0001-0001,nzd0001,359.037136,3097.737936,0.968728,NaN,NaN,NaN,-0.193058,194.0,166.0,0.004690,17.523830,479.691514,21.901861,329.329626,None,"LINESTRING (1596744.903 6190264.499, 1596729.1..."
95052,nzd0001-0002,nzd0001,359.037136,2997.737936,0.937456,0.060,0.0478,0.0776,-0.280504,194.0,169.0,0.012589,15.783007,369.596582,19.224895,335.326568,None,"LINESTRING (1596827.431 6190265.908, 1596811.6..."
95053,nzd0001-0003,nzd0001,359.037136,2897.737936,0.906184,NaN,NaN,NaN,-0.319317,194.0,171.0,0.020475,14.218951,297.704034,17.254102,343.806416,None,"LINESTRING (1596909.959 6190267.317, 1596894.2..."
95054,nzd0001-0004,nzd0001,359.037136,2797.737936,0.874912,0.055,0.0450,0.0664,-0.385838,194.0,172.0,0.032657,13.279462,274.092958,16.555753,356.283423,None,"LINESTRING (1596992.486 6190268.724, 1596976.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147262,nzd0564-0046,nzd0564,NaN,NaN,NaN,0.110,0.1000,0.1279,0.122686,989.0,772.0,0.012423,6.278019,68.394524,8.270098,223.588670,None,"LINESTRING (1630436.415 5440463.658, 1630195.7..."
147263,nzd0564-0047,nzd0564,NaN,NaN,NaN,0.090,0.0846,0.0955,0.222498,989.0,745.0,0.037752,6.548643,72.357463,8.506319,221.910745,None,"LINESTRING (1630500.321 5440511.784, 1630259.6..."
147264,nzd0564-0048,nzd0564,NaN,NaN,NaN,0.080,0.0712,0.0909,0.098210,989.0,773.0,0.005900,6.922215,93.340801,9.661304,230.537534,None,"LINESTRING (1630576.662 5440571.499, 1630309.8..."
147265,nzd0564-0049,nzd0564,NaN,NaN,NaN,0.095,0.0893,0.1021,0.142179,989.0,745.0,0.013291,6.946543,86.459034,9.298335,223.251099,None,"LINESTRING (1630636.335 5440624.99, 1630369.27..."


In [16]:
def create_transects(line, spacing=80, transect_length=400):
    transects = []
    distances = np.arange(0, line.length, spacing)

    for distance in distances:
        # Find point along the average line
        point = line.interpolate(distance)
        
        # Find the direction of the line at this point (tangent direction)
        nearest_point_ahead = line.interpolate(min(distance + 1e-6, line.length))
        direction = np.arctan2(nearest_point_ahead.y - point.y, nearest_point_ahead.x - point.x)
        
        # Rotate 90 degrees (perpendicular) and extend to create a transect
        transect = LineString(reversed([
            Point(
                point.x - transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y - transect_length / 2 * np.sin(direction + np.pi / 2)
            ),
            Point(
                point.x + transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y + transect_length / 2 * np.sin(direction + np.pi / 2)
            )
        ]))
        
        transects.append(transect)
    
    transects = gpd.GeoDataFrame(geometry=transects, crs=2193).to_crs(4326)
    return transects

all_new_transects = []
for new_shoreline in new_shorelines.to_crs(2193).itertuples():
    new_transects = create_transects(new_shoreline.geometry)
    new_transects["id"] = new_shoreline.id + "-" + new_transects.index.astype(str).str.pad(4, fillchar="0")
    new_transects["site_id"] = new_shoreline.id
    all_new_transects.append(new_transects)

all_new_transects = gpd.GeoDataFrame(pd.concat(all_new_transects).reset_index(drop=True), crs=4326)
display(all_new_transects)
m = all_new_transects.explore()
new_shorelines.explore(m=m)
new_polys.boundary.explore(m=m)
gpd.GeoSeries(all_new_transects.geometry.apply(lambda line: Point(line.coords[0])), crs=all_new_transects.crs).explore(m=m, color="red", name="transect start")
print("Make sure the origin is inland")
m

,geometry,id,site_id
0,"LINESTRING (173.01586 -41.01938, 173.02062 -41...",nzd0565-0000,nzd0565
1,"LINESTRING (173.01603 -41.01836, 173.02068 -41...",nzd0565-0001,nzd0565
2,"LINESTRING (173.01697 -41.01689, 173.02054 -41...",nzd0565-0002,nzd0565
3,"LINESTRING (173.48015 -41.15877, 173.47933 -41...",nzd0566-0000,nzd0566
4,"LINESTRING (173.47921 -41.15889, 173.4784 -41....",nzd0566-0001,nzd0566
...,...,...,...
231,"LINESTRING (173.04709 -40.94777, 173.05181 -40...",nzd0579-0000,nzd0579
232,"LINESTRING (173.04699 -40.94685, 173.05174 -40...",nzd0579-0001,nzd0579
233,"LINESTRING (173.04703 -40.94597, 173.05177 -40...",nzd0579-0002,nzd0579
234,"LINESTRING (173.04718 -40.94513, 173.05187 -40...",nzd0579-0003,nzd0579


Make sure the origin is inland


In [17]:
pd.concat((transects, new_transects)).to_file("transects_extended.geojson", driver="GeoJSON")